# Example 1: Basic Human-in-the-Loop Text Approval

## Problem Statement
Create a simple LangGraph that generates text content (like social media posts or emails) and requires human approval before the content is considered "published" or finalized. This demonstrates the most basic human-in-the-loop pattern where a human acts as a quality gate.

## Features
- Generate text content using an LLM
- Present content to human for approval/rejection
- Handle human feedback and regenerate if needed
- Basic error handling and validation

In [ ]:
# Install required packages (run this cell if packages are not installed)
# !pip install langchain langgraph langchain-openai pytest

In [ ]:
import os
from typing import TypedDict, Annotated, List, Literal
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
import json
from datetime import datetime

# Set up OpenAI API key (make sure this is set in your environment)
# os.environ["OPENAI_API_KEY"] = "your-api-key-here"

print("📚 Basic Human-in-the-Loop Text Approval Example")
print("=" * 50)

## Step 1: Define the State
The state keeps track of the conversation, approval status, and any feedback from the human.

In [ ]:
class BasicApprovalState(TypedDict):
    messages: Annotated[List, add_messages]
    generated_content: str
    approved: bool
    feedback: str
    attempt_count: int
    max_attempts: int

print("✅ State defined with fields:")
print("   - messages: conversation history")
print("   - generated_content: the AI-generated text")
print("   - approved: human approval status")
print("   - feedback: human feedback for improvements")
print("   - attempt_count: number of generation attempts")
print("   - max_attempts: maximum allowed attempts")

## Step 2: Define the Graph Nodes
Each node represents a step in the approval workflow.

In [ ]:
def generate_content(state: BasicApprovalState) -> BasicApprovalState:
    """Generate text content using LLM."""
    print("🤖 Generating content...")
    
    try:
        messages = state["messages"]
        attempt = state.get("attempt_count", 0) + 1
        feedback = state.get("feedback", "")
        
        # Build system message based on attempt and feedback
        system_content = "You are a helpful content creator. Generate engaging, appropriate content based on the user's request."
        
        if attempt > 1 and feedback:
            system_content += f"\n\nPrevious attempt was not approved. User feedback: {feedback}\nPlease improve the content based on this feedback."
        
        system_msg = SystemMessage(content=system_content)
        full_messages = [system_msg] + messages
        
        llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
        response = llm.invoke(full_messages)
        
        content = response.content
        print(f"📝 Generated content (attempt {attempt}): {content[:100]}...")
        
        return {
            "generated_content": content,
            "attempt_count": attempt,
            "messages": [response]
        }
        
    except Exception as e:
        print(f"❌ Error generating content: {e}")
        return {
            "generated_content": f"Error generating content: {str(e)}",
            "attempt_count": state.get("attempt_count", 0) + 1
        }


def request_human_approval(state: BasicApprovalState) -> BasicApprovalState:
    """Request human approval for the generated content."""
    print("👤 Requesting human approval...")
    
    try:
        content = state["generated_content"]
        attempt = state.get("attempt_count", 1)
        
        print(f"\n{'='*60}")
        print(f"📋 CONTENT APPROVAL REQUEST (Attempt {attempt})")
        print(f"{'='*60}")
        print(f"Generated Content:\n{content}")
        print(f"{'='*60}")
        
        while True:
            decision = input("\nDo you approve this content? (yes/no/feedback): ").strip().lower()
            
            if decision in ['yes', 'y', 'approve']:
                print("✅ Content approved!")
                return {
                    "approved": True,
                    "feedback": ""
                }
            elif decision in ['no', 'n', 'reject']:
                feedback = input("Please provide feedback for improvement: ").strip()
                print(f"❌ Content rejected. Feedback: {feedback}")
                return {
                    "approved": False,
                    "feedback": feedback
                }
            elif decision == 'feedback':
                feedback = input("Please provide your feedback: ").strip()
                print(f"📝 Feedback received: {feedback}")
                return {
                    "approved": False,
                    "feedback": feedback
                }
            else:
                print("Please enter 'yes', 'no', or 'feedback'")
                
    except Exception as e:
        print(f"❌ Error in approval process: {e}")
        return {
            "approved": False,
            "feedback": f"Error in approval process: {str(e)}"
        }


def finalize_content(state: BasicApprovalState) -> BasicApprovalState:
    """Finalize the approved content."""
    print("📋 Finalizing approved content...")
    
    try:
        content = state["generated_content"]
        timestamp = datetime.now().isoformat()
        
        final_message = AIMessage(
            content=f"Content approved and finalized at {timestamp}:\n\n{content}"
        )
        
        print("✅ Content has been approved and finalized!")
        print(f"📅 Timestamp: {timestamp}")
        
        return {
            "messages": [final_message]
        }
        
    except Exception as e:
        print(f"❌ Error finalizing content: {e}")
        return {
            "messages": [AIMessage(content=f"Error finalizing content: {str(e)}")]
        }

print("✅ Nodes defined: generate_content, request_human_approval, finalize_content")

## Step 3: Define Routing Logic
The routing logic determines the flow through the graph based on approval status and attempt limits.

In [ ]:
def should_continue(state: BasicApprovalState) -> Literal["approve", "regenerate", "end"]:
    """Determine next step based on approval status and attempt count."""
    approved = state.get("approved", False)
    attempt_count = state.get("attempt_count", 0)
    max_attempts = state.get("max_attempts", 3)
    
    if approved:
        print("🔀 Routing to finalization")
        return "approve"
    elif attempt_count >= max_attempts:
        print(f"🔀 Max attempts ({max_attempts}) reached, ending")
        return "end"
    else:
        print(f"🔀 Routing to regeneration (attempt {attempt_count + 1})")
        return "regenerate"


def handle_max_attempts(state: BasicApprovalState) -> BasicApprovalState:
    """Handle case when maximum attempts are reached."""
    print("⚠️ Maximum attempts reached without approval")
    
    message = AIMessage(
        content="Maximum attempts reached. Content could not be approved. Please try with different requirements."
    )
    
    return {
        "messages": [message]
    }

print("✅ Routing logic defined")

## Step 4: Build the Graph
Construct the LangGraph with nodes and edges.

In [ ]:
def build_basic_approval_graph():
    """Build and compile the basic approval graph."""
    print("🏗️ Building basic approval graph...")
    
    workflow = StateGraph(BasicApprovalState)
    
    # Add nodes
    workflow.add_node("generate", generate_content)
    workflow.add_node("approval", request_human_approval)
    workflow.add_node("finalize", finalize_content)
    workflow.add_node("max_attempts", handle_max_attempts)
    
    # Set entry point
    workflow.set_entry_point("generate")
    
    # Add edges
    workflow.add_edge("generate", "approval")
    
    # Conditional routing from approval
    workflow.add_conditional_edges(
        "approval",
        should_continue,
        {
            "approve": "finalize",
            "regenerate": "generate",
            "end": "max_attempts"
        }
    )
    
    # End nodes
    workflow.add_edge("finalize", END)
    workflow.add_edge("max_attempts", END)
    
    # Compile the graph
    graph = workflow.compile()
    
    print("✅ Graph compiled successfully!")
    print("   Flow: generate → approval → [approve/regenerate/end] → finalize/max_attempts")
    
    return graph

# Build the graph
basic_graph = build_basic_approval_graph()

## Step 5: Test the Graph
Run the graph with sample inputs to demonstrate the human-in-the-loop functionality.

In [ ]:
def run_basic_example(user_request: str, max_attempts: int = 3):
    """Run the basic approval example."""
    print(f"\n{'='*60}")
    print(f"🚀 Running Basic Approval Example")
    print(f"Request: {user_request}")
    print(f"Max attempts: {max_attempts}")
    print(f"{'='*60}")
    
    initial_state = {
        "messages": [HumanMessage(content=user_request)],
        "generated_content": "",
        "approved": False,
        "feedback": "",
        "attempt_count": 0,
        "max_attempts": max_attempts
    }
    
    try:
        final_state = None
        for state in basic_graph.stream(initial_state):
            final_state = state
        
        print("\n" + "="*60)
        print("📋 FINAL RESULT")
        print("="*60)
        
        if final_state:
            last_state = list(final_state.values())[-1]
            if "messages" in last_state and last_state["messages"]:
                final_message = last_state["messages"][-1]
                print(f"Result: {final_message.content}")
            
            # Show statistics
            attempts = last_state.get("attempt_count", 0)
            approved = last_state.get("approved", False)
            print(f"\n📊 Statistics:")
            print(f"   - Total attempts: {attempts}")
            print(f"   - Final status: {'✅ Approved' if approved else '❌ Not approved'}")
        
        print("\n✅ Example completed successfully!")
        return final_state

    except Exception as e:
        print(f"❌ Error running example: {e}")
        return None

# Example usage
print("📝 Example ready to run!")
print("\nTo test the graph, run:")
print('run_basic_example("Write a social media post about the benefits of reading books")')

In [ ]:
# Run an example (uncomment to test)
result = run_basic_example("Write a social media post about the benefits of reading books")

## Step 6: Unit Tests
Test the core functionality of the graph components.

In [ ]:
import pytest
from unittest.mock import patch, MagicMock

def test_generate_content_basic():
    """Test basic content generation."""
    state = {
        "messages": [HumanMessage(content="Write a hello world message")],
        "attempt_count": 0,
        "feedback": ""
    }
    
    with patch('langchain_openai.ChatOpenAI') as mock_llm_class:
        mock_llm = MagicMock()
        mock_response = MagicMock()
        mock_response.content = "Hello, World! This is a test message."
        mock_llm.invoke.return_value = mock_response
        mock_llm_class.return_value = mock_llm
        
        result = generate_content(state)
        
        assert "generated_content" in result
        assert result["generated_content"] == "Hello, World! This is a test message."
        assert result["attempt_count"] == 1
        
    print("✅ test_generate_content_basic passed")

def test_routing_logic():
    """Test routing logic for different scenarios."""
    # Test approved scenario
    approved_state = {"approved": True, "attempt_count": 1, "max_attempts": 3}
    assert should_continue(approved_state) == "approve"
    
    # Test max attempts reached
    max_attempts_state = {"approved": False, "attempt_count": 3, "max_attempts": 3}
    assert should_continue(max_attempts_state) == "end"
    
    # Test regeneration needed
    regenerate_state = {"approved": False, "attempt_count": 1, "max_attempts": 3}
    assert should_continue(regenerate_state) == "regenerate"
    
    print("✅ test_routing_logic passed")

def test_error_handling():
    """Test error handling in content generation."""
    state = {
        "messages": [HumanMessage(content="Test message")],
        "attempt_count": 0
    }
    
    with patch('langchain_openai.ChatOpenAI') as mock_llm_class:
        mock_llm_class.side_effect = Exception("API Error")
        
        result = generate_content(state)
        
        assert "Error generating content" in result["generated_content"]
        assert result["attempt_count"] == 1
        
    print("✅ test_error_handling passed")

# Run the tests
print("🧪 Running unit tests...")
test_generate_content_basic()
test_routing_logic()
test_error_handling()
print("\n✅ All unit tests passed!")

## Explanation

### Graph Structure
The basic approval graph follows this flow:
1. **Generate**: AI creates content based on user request
2. **Approval**: Human reviews and approves/rejects with feedback
3. **Decision**: Route based on approval status and attempt count
4. **Finalize/Retry**: Either finalize approved content or regenerate with feedback

### Human Intervention Point
The human intervention occurs in the `request_human_approval` node where:
- Generated content is displayed to the user
- User can approve, reject, or provide specific feedback
- Feedback is incorporated into subsequent generation attempts

### Graph Adaptation
The graph adapts to human input by:
- Incorporating feedback into the system prompt for regeneration
- Tracking attempt count to prevent infinite loops
- Routing to different outcomes based on approval decisions

### Key Features
- **Error Handling**: Graceful handling of API errors and invalid inputs
- **Attempt Limiting**: Prevents infinite regeneration loops
- **Feedback Integration**: Uses human feedback to improve subsequent attempts
- **State Tracking**: Maintains conversation history and approval status